In [1]:
from geopy.geocoders import GoogleV3
import numpy as np
import geocoder
import pandas as pd
import base64
from geojson import Feature, FeatureCollection, Point

In [2]:
%store -r google_maps_API_Key
geolocator = GoogleV3(api_key=google_maps_API_Key)

In [3]:
df = pd.read_csv('so_fla_rappers.csv')

In [4]:
df

,rapper,address,display_address,price,date_bought,image_url,attribution_url
0,"Sean ""P. Diddy"" Combs","1 Star Island Drive, Miami Beach, Fl 33139","1 Star Island Drive, Miami Beach",$35 million,July 2021,https://en.wikipedia.org/wiki/Sean_Combs#/medi...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
1,"Sean ""P. Diddy"" Combs","2 Star Island Drive, Miami Beach, Fl 33139","1 Star Island Drive, Miami Beach",$15 million,November 2003,https://en.wikipedia.org/wiki/Sean_Combs#/medi...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
2,DJ Khaled,"4609 Pine Tree Drive, Miami Beach, FL 33140","4609 Pine Tree Drive, Miami Beach",$21.7 million,September 2018,https://en.wikipedia.org/wiki/DJ_Khaled#/media...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
3,Lil Wayne,"6480 Allison Road, Miami Beach, FL 33141","4609 Pine Tree Drive, Miami Beach",$16.7 million,November 2018,https://en.wikipedia.org/wiki/Lil_Wayne#/media...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
4,Lil Pump,"1510 Bay Drive, Miami Beach, FL 33141","1510 Bay Drive, Miami Beach",$4.6 million,July 2019,https://en.wikipedia.org/wiki/Lil_Pump#/media/...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
5,Trippie Redd,"5600 Southwest 136th Avenue, Southwest Ranches...","5600 Southwest 136th Avenue, Southwest Ranches",$7.5 million,May 2022,https://en.wikipedia.org/wiki/Trippie_Redd#/me...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
6,Rick Ross,"16800 Bershire Court, Southwest Ranches, FL 33331","16800 Bershire Court, Southwest Ranches",$3.5 million,March 2021,https://en.wikipedia.org/wiki/Rick_Ross,"<a href=""https://commons.wikimedia.org/wiki/Fi..."
7,Kodak Black,"10031 Vestal Place, Coral Springs","10031 Vestal Place, Coral Springs",$1.8 million,February 2022,https://en.wikipedia.org/wiki/Kodak_Black#/med...,"<a href=""https://commons.wikimedia.org/wiki/Fi..."


In [5]:
df['loc'] = df['address'].apply(geolocator.geocode, timeout=10)

In [6]:
df['loc']

0    (1 W Star Island Dr, Miami Beach, FL 33139, US...
1    (2 W Star Island Dr, Miami Beach, FL 33139, US...
2    (4609 Pine Tree Dr, Miami Beach, FL 33140, USA...
3    (6480 Allison Rd, Miami Beach, FL 33141, USA, ...
4    (1510 Bay Dr, Miami Beach, FL 33141, USA, (25....
5    (5600 SW 136th Ave, Southwest Ranches, FL 3333...
6    (16800 Berkshire Ct, Southwest Ranches, FL 333...
7    (10031 Vestal Pl, Coral Springs, FL 33071, USA...
Name: loc, dtype: object

In [7]:
df['point'] = df['loc'].apply(lambda loc: tuple(loc.point) if loc else None)

In [8]:
df[['lat','lon','altitude']] = pd.DataFrame(df['point'].to_list(),index=df.index)

In [9]:
df['image_img_test'] = ' '

In [10]:
df.at[0,'image_img_test']=('<img src="Sean_Combs_2010.jpg" width="256" height="174">')
df.at[1,'image_img_test']=('<img src="Sean_Combs_2010.jpg" width="256" height="174">')
df.at[2,'image_img_test']=('<img src="DJ_Khaled_2012.jpg" width="256" height="188">')
df.at[3,'image_img_test']=('<img src="LIL_WAYNE.jpg" width="256" height="252">')
df.at[4,'image_img_test']=('<img src="Lil_Pump.jpg" width="256" height="190">')
df.at[5,'image_img_test']=('<img src="Tripplie_Redd.jpg" width="256" height="170">')
df.at[6,'image_img_test']=('<img src="Rick_Ross.jpg" width="256" height="199">')
df.at[7,'image_img_test']=('<img src="Kodak_Black.jpg" width="256" height="136">')

In [11]:
df.columns

Index(['rapper', 'address', 'display_address', 'price', 'date_bought',
       'image_url', 'attribution_url', 'loc', 'point', 'lat', 'lon',
       'altitude', 'image_img_test'],
      dtype='object')

In [18]:
import folium
from folium.plugins import MarkerCluster
import textwrap

m = folium.Map(location=df[["lat", "lon"]].mean().to_list(), zoom_start=9.75)
title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Here\'s where big time rappers bought homes in South Florida')

# marker_cluster = MarkerCluster().add_to(m)

for i,r in df.iterrows():
    location = (r["lat"], r["lon"])
    info = (r["rapper"], r["display_address"], r["price"], r['date_bought'], r['image_img_test'], r['attribution_url'])
    info = list(info)
    new_line = '<br>'
    bold_start = '<strong>'
    bold_end = '</strong>'
    italic_start = '<em>'
    italic_end = '</em>'
    text = f'{info[-2]}{new_line} \
    {italic_start}{info[-1]}{italic_end}{new_line}\
    Rapper: {bold_start}{info[0]}{bold_end}{new_line} \
    Address: {bold_start}{info[1]}{bold_end}{new_line}Price: \
    {bold_start}{info[2]}{bold_end}{new_line}Date bought: {bold_start}{info[3]}{bold_end}'
    text = text.replace(",",",<br>")
    folium.Marker(location=location,
                      tooltip=text)\
    .add_to(m)

m.get_root().html.add_child(folium.Element(title_html))

m

In [19]:
m.save('index.html')